<a href="https://colab.research.google.com/github/fovi-llc/unsloth-notebooks/blob/main/nb/Mistral_Small_22B_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
# %%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install "unsloth==2025.2.9" vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

Mount GDrive for persistence

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Data/Unsloth

import datetime
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_dir = f"mistral-grpo_{now}"
!mkdir {run_dir}
%cd {run_dir}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Data/Unsloth
/content/drive/MyDrive/Data/Unsloth/mistral-grpo_20250215-010318


In [2]:
!pwd
hf_hub_path = f"jimwhite/{run_dir}"
hf_hub_path

/content/drive/MyDrive/Data/Unsloth/mistral-grpo_20250215-010318


### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-15 01:03:32 __init__.py:190] Automatically detected platform cuda.


Load up `Mistral-Small-24B-Instruct`, and set parameters

In [4]:
from unsloth import is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
import re
from datasets import load_dataset, Dataset
max_seq_length = 8096 # Supports RoPE Scaling interally, so choose any!

lora_rank = 64
lora_alpha = 64

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Small-24B-Instruct-2501",
    max_seq_length = max_seq_length,
    dtype = None,
    fast_inference = True,
    max_lora_rank = lora_rank,
    # gpu_memory_utilization = 0.5,
    gpu_memory_utilization = 0.8,
    load_in_4bit = True,
    # load_in_8bit = True, there is only unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit no -8bit
)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = 0,
    # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth 2025.2.9: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/mistral-small-24b-instruct-2501-unsloth-bnb-4bit with actual GPU utilization = 79.16%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8096. Num Sequences = 256.
Unsloth: vLLM's KV Cache can use up to 14.87 GB. Also swap space = 6 GB.
INFO 02-15 01:04:28 config.py:542] This model supports multiple tasks: {'score', 'reward', 'generate', 'classify', 'embed'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes co

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 02-15 01:04:42 model_runner.py:1115] Loading model weights took 13.9475 GB
INFO 02-15 01:04:42 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 02-15 01:04:47 worker.py:267] Memory profiling takes 4.36 seconds
INFO 02-15 01:04:47 worker.py:267] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.79) = 31.31GiB
INFO 02-15 01:04:47 worker.py:267] model weights take 13.95GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 2.05GiB; the rest of the memory reserved for KV Cache is 15.22GiB.
INFO 02-15 01:04:47 executor_base.py:110] # CUDA blocks: 6234, # CPU blocks: 2457
INFO 02-15 01:04:47 executor_base.py:115] Maximum concurrency for 8096 tokens per request: 12.32x
INFO 02-15 01:04:52 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:49<00:00,  1.40s/it]

INFO 02-15 01:05:41 model_runner.py:1562] Graph capturing finished in 49 secs, took 1.22 GiB
INFO 02-15 01:05:41 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 59.61 seconds


tokenizer_config.json:   0%|          | 0.00/200k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/21.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Unsloth 2025.2.9 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [5]:
# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [6]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    # optim = "paged_adamw_8bit",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 8,
    max_prompt_length = 256,
    # max_completion_length = 200,
    max_completion_length = 300,
    # num_train_epochs = 1,
    # max_steps = 700,
    max_steps = 250,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [7]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 369,623,040


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<reasoning>
1. Calculate the cost of the first 10 tickets without discount: 10 tickets * $40/ticket = $400
2. Calculate the discount for the remaining 2 tickets: 5% of $40 = $40 * 0.05 = $2
3. Calculate the cost of the remaining 2 tickets with discount: $40 - $2 = $38 per ticket
4. Calculate the total cost for the remaining 2 tickets: $38 * 2 = $76
5. Add the cost of the first 10 tickets and the remaining 2 tickets to get the total cost: $400 + $76 = $476
</reasoning>
<answer>
Mr. Benson paid $476 in all.
</answer> 
Extracted:
Mr. Benson paid $476 in all.


Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,-0.111250,0.064694,198.625000,0.000323
2,0.000000,-0.084250,0.204563,237.250000,0.000363
3,0.000000,-0.338250,0.199617,247.625000,0.000248
4,0.000000,0.128000,0.837104,191.625000,0.000324
5,0.000000,-0.068375,0.095922,191.125000,0.000283
6,0.000000,-0.265625,0.104511,272.625000,0.000280
7,0.000000,-0.215875,0.127338,235.250000,0.000183
8,0.000000,0.001625,0.221288,135.625000,0.000485
9,0.000000,0.119625,0.942284,204.875000,0.000341
10,0.000000,-0.310750,0.232032,274.500000,0.000407


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
<reasoning>
To determine how much more the monthly payment on the house is compared to the trailer, we need to calculate the monthly payments for both the house and the trailer. For simplicity, let's assume that the interest rates for both loans are the same and the loans are simple interest loans.

Let's denote the principal amount for the house as P_{house} = $480,000 and for the trailer as P_{trailer} = $120,000. The number of years for the loan is 20 years which is 240 months.

The formula for the monthly payment (M) for a simple interest loan is:

\[ P = Ih + Mh \]
\[ M = \frac{P \times \frac{i}{12}}{n} \]
Where:
- \( P \) is the principal amount.
- \( i \) is the annual interes

TrainOutput(global_step=250, training_loss=0.009626429639756679, metrics={'train_runtime': 4513.5859, 'train_samples_per_second': 0.055, 'train_steps_per_second': 0.055, 'total_flos': 0.0, 'train_loss': 0.009626429639756679})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [27]:
import textwrap

text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

print()
print(textwrap.fill(output, width=80, replace_whitespace=False))
output

Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.30s/it, est. speed input: 14.71 toks/s, output: 32.27 toks/s]

Calculating the value of π (pi) can be done using various methods, ranging from
simple approximations to complex algorithms. One of the most well-known methods
is the Gregory-Leibniz series, but it converges very slowly. A more efficient
method is the Gauss-Legendre algorithm, which converges much faster.

Here's a
simple implementation of the Gregory-Leibniz series to approximate π:

\[ \pi =
4 \left( 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} -
\frac{1}{11} + \cdots \right) \]

Let's write a Python code snippet to calculate
π using this series:

```python
def calculate_pi(num_terms):
pi_approximation = 0
    for i in range(num_terms):
        term = 4 * ((-1) **
i) / (2 * i + 1)
        pi_approximation += term
    return pi_approximation

#
Calculate π with a certain number of terms
num_terms = 10000
pi_value =
calculate_pi(num_terms)
print(f"Approximation of π with {num_terms} terms:
{pi_value}")
```

This code will give you an approximation of π using the first
`num

'Calculating the value of π (pi) can be done using various methods, ranging from simple approximations to complex algorithms. One of the most well-known methods is the Gregory-Leibniz series, but it converges very slowly. A more efficient method is the Gauss-Legendre algorithm, which converges much faster.\n\nHere\'s a simple implementation of the Gregory-Leibniz series to approximate π:\n\n\\[ \\pi = 4 \\left( 1 - \\frac{1}{3} + \\frac{1}{5} - \\frac{1}{7} + \\frac{1}{9} - \\frac{1}{11} + \\cdots \\right) \\]\n\nLet\'s write a Python code snippet to calculate π using this series:\n\n```python\ndef calculate_pi(num_terms):\n    pi_approximation = 0\n    for i in range(num_terms):\n        term = 4 * ((-1) ** i) / (2 * i + 1)\n        pi_approximation += term\n    return pi_approximation\n\n# Calculate π with a certain number of terms\nnum_terms = 10000\npi_value = calculate_pi(num_terms)\nprint(f"Approximation of π with {num_terms} terms: {pi_value}")\n```\n\nThis code will give you an

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [9]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [29]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

print()
print(textwrap.fill(output, width=80, replace_whitespace=False))
output

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it, est. speed input: 20.14 toks/s, output: 18.92 toks/s]


<reasoning>
3.14159
</reasoning>
<answer>
3.14159
</answer>


'<reasoning>\n3.14159\n</reasoning>\n<answer>\n3.14159\n</answer>'

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [31]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged(hf_hub_path, tokenizer, save_method = "merged_16bit", token = "")

# Just LoRA adapters
if True: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if True: model.push_to_hub_merged(hf_hub_path, tokenizer, save_method = "lora", token = "")

# Merge to 4bit
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if True: model.push_to_hub_merged(hf_hub_path, tokenizer, save_method = "merged_4bit", token = "")

RuntimeError: Unsloth: Merging into 4bit will cause your model to lose accuracy if you plan
to merge to GGUF or others later on. I suggest you to do this as a final step
if you're planning to do multiple saves.
If you are certain, change `save_method` to `merged_4bit_forced`.

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf(hf_hub_path, tokenizer)

# Save to 16bit GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf(hf_hub_path, tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf(hf_hub_path, tokenizer, quantization_method = "q4_k_m")

# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        hf_hub_path, # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 44.46 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [03:25<00:00,  5.13s/it]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting mistral model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/drive/MyDrive/Data/Unsloth/mistral-grpo_20250215-010318/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00010.safetensors'
INFO:hf-to-gguf:token_e

100%|██████████| 40/40 [03:20<00:00,  5.00s/it]


Unsloth: Saving tokenizer...

JPW: Looks like default_system_message is still the base model's which surely isn't correct and should be the system message used for QRPO.

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
